# Project Lab #1: Reading
## Spacecraft flybys of planets and moons.

<div class="alert alert-block alert-danger"> 
    <b>
        You need to read this notebook carefully to understand what is going on, and to be able to plan your algorithm for computing the spacecraft acceleration, velocity and position during the flyby.
    </b>
</div>

<!-- ###  leave the cell below unedited

- it formats the equations nicely with equation numbers etc ...
- if the equation numbering doesn't appear for you try refreshing your browser page and/or execute the cell below.  I sometimes have to do this a couple of times for it to work. -->

<script type="text/javascript" id="MathJax-script" defer
  src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-svg.js">
</script>

## 1.  Background and Motivation
This project lab is motivated by the use of flybys of planets / moons in spacecraft missions to alter a spacecraft’s trajectory (flight path). Planetary flybys can be used to not only change the direction of a spacecraft’s
motion, but to also alter its speed. Such flybys are often referred to as “gravity assists” because the change to
the spacecraft’s motion is enabled by the gravitational force of the planet on the spacecraft. For example, missions to the outer solar system typically use the gravity of Jupiter to provide a “boost” to the spacecraft’s speed,
such that the spacecraft can actually make it to the outer solar system. In contrast, missions to the inner solar system typically use the gravity field of one or more planets to slow the spacecraft down enough that it can
go into orbit around the desired planet. A good resource describing planetary gravity assists can be found <u>[here](https://www.planetary.org/articles/20130926-gravity-assist)</u>.

We will construct a simplified planetary flyby. The simplifications are that we assume an otherwise “straight-line” path of the spacecraft (rather than starting with an elliptical orbit around the sun, or a hyberbolic trajectory into the outer solar system), and we assume that the planet is stationary during the flyby (in practice the planet is of course in an elliptical orbit around the Sun). We will implement a numerical integration algorithm using python to compute a spacecraft’s acceleration, velocity and position as it flys by a planet. This will also demonstrate how you can use python to produce numerical solutions to problems for which there is no simple analytical solution.  The physics that is needed is straightforward first year physics - Newton's second law of motion and Newton's law of gravity, reviewed for this problem below.


___ 

## 2. The Physics

**Figure 1: Example of flyby trajectory**
<img src="./fig1.png" width="500">

We have a spacecraft which is about to fly past a planet or moon. We will simplify the problem and look at it in a Cartesian ($x,y$) geometry. The origin of our coordinate system is at the center of the planet (see **Figure 1**). The spacecraft motion is such that if the planet was not there, the spacecraft would be flying with constant velocity in the positive y-direction. Our spacecraft starts out with this velocity at some distance from the planet (bottom of Figure 1).  Far away from the planet the spacecraft has constant velocity (remember Newton’s second law: no net force on the spacecraft implies constant momentum ($m \vec{v}$), and so constant velocity). As the spacecraft approaches the planet, it experiences a gravitational force due to the planet. 

### 2.1 The gravitational force on the spacecraft from the planet

<!-- <div class="alert alert-block alert-warning"> -->
    
The first part of the problem is to know how to calculate the gravitational acceleration on the spacecraft at any position in space.  Accordingly, the first part of the lab will be to write a function `grav_acc` that will take a single spacecraft position, the planet mass and will compute the gravitational acceleration on the spacecraft. This is a vector quantity and you will resolve the acceleration into its two components using the physics below.

If the spacecraft is at a position $\vec{s}$, Newton’s law of gravitation tells us that the magnitude of the force on the spacecraft, $F_{sc}$, will be

$$\begin{equation}
F_{s c}=\frac{G M_{p} m_{s c}}{s^{2}} \tag{1}
\end{equation}$$

where $s$ is the distance of the spacecraft **from the center of the planet**.  The position vector, $\vec{s}$ has both $x$ and $y$ components, so the vector, $\vec{s}$ is given by $\vec{s}=s_{x} \hat{x}+s_{y} \hat{y}$, and its magnitude, $s$ is given by $s=\sqrt{\left(s_{x}^{2}+s_{y}^{2}\right)}$. 
      
- NOTE: $\hat{x}$ and $\hat{y}$ are unit vectors in the $x$ and $y$ directions, you might have seen these also written as ${i}$ and ${j}$ in math/physics.   
    
$M_p$ is the mass of the planet, $m_{sc}$ is the mass of the spacecraft, and $G$ is the gravitational constant (= $6.67 \times 10^{−11}$ m$^3$ kg$^{−1}$ s$^{2}$)  


The spacecraft will thus experience an instantaneous acceleration in a direction toward the center of the planet. We use  $\vec{a}$ to denote the acceleration vector:   

$$\begin{equation}
\vec{a}=\frac{\vec{F}_{s c}}{m_{s c}} \tag{2}
\end{equation}$$
    
- NOTE: the force depends on the spacecraft mass but the acceleration does not.  Substitute Equation 1 into Equation 2 to see that this is the case. 

The instantaneous acceleration vector, $\vec{a}$, can be resolved into components along the x- and y- axes (see **Figure 2**).
  
    
$$\begin{equation} a_{x}=a \sin \beta \tag{3} \end{equation}$$
$$\begin{equation} a_{y}=a \cos \beta \tag{4} \end{equation}$$


where $a$ is the magnitude of the acceleration vector, $a=\frac{G M_{p}}{s^{2}}$. $\beta$ is the angle between the acceleration vector and the $\hat y$ direction and can easily be found from the instantaneous position vector. $\sin \beta$ and $\cos \beta$ are given by:

$$\begin{equation} \sin \beta=\frac{-s_{x}}{s} \tag{5} \end{equation}$$
$$\begin{equation} \cos \beta=\frac{-s_{y}}{s} \tag{6} \end{equation}$$


The minus signs in (5) and (6) ensure that $a_x$ and $a_y$ are positive when $s_x$ and $s_y$ are respectively negative and vice-versa (i.e., in the figure, the spacecraft acceleration is towards the planet).  Because the origin is at the center of the planet negative values for $s_x$ correspond to points in the left half of Figure 1 and negative values for $s_y$ correspond to points in the bottom half of Figure 2.

<!-- </div> -->

**Figure 2: Instantaneous acceleration AND velocity**
<img src="./fig2.png" width="500">


### 2.2 How this force changes the spacecraft velocity and affects the spacecraft's future position

The spacecraft velocity is changed due to this acceleration; both the speed and the direction of the velocity vector change. This is easiest to see at the beginning of the spacecraft’s trajectory. Initially the spacecraft has a speed
$v_{y}^{\text {initial }}$ in the y-direction. Remember from Newton's second law of motion that force equals mass * acceleration = change in momentum.  i.e., 

$$ \begin{equation} F = m a = \frac{d (mv)}{dt} =  m \frac{d (v)}{dt}. \tag{7} \end{equation}$$

So $$ \begin{equation} a = dv/dt \tag{8} \end{equation}$$

or for small time intervals, $\Delta t$, we can write

$$ \begin{equation} a = \frac{\Delta v}{\Delta t} \tag{9} \end{equation}$$

In other words, the acceleration means that after a small time, $\Delta t$, the component of velocity along the y-axis is increased by $a_{y} \Delta t$, so that the new speed after a time $\Delta t$, $v_{y}^{\text {new }}$ is given by $v_{y}^{\text {new }}=v_{y}^{\text {initial }}+a_{y} \Delta t$. Similarly, the component of velocity in the x-direction is changed to $v_{x}^{\text {new }}=v_{x}^{\text {initial }}+a_{x} \Delta t$. In our problem, $v_{x}^{\text {initial }}=0$, but the new velocity has non-zero $x$- and $y$- components, i.e. its direction has been changed as well as the net speed. 

 
We can generalize this so that at any point along the spacecraft’s trajectory we can compute the acceleration due to the gravitational field of the planet, along with the resulting change in velocity and position over some small time increment $\Delta t$. The equations for each component of velocity and position are as follows:


$$\begin{equation} v_{x}^{i+1}=v_{x}^{i}+\Delta v_{x}^{i+1} \tag{10}\end{equation}$$

$$\begin{equation} v_{y}^{i+1}=v_{y}^{i}+\Delta v_{y}^{i+1} \tag{11}\end{equation}$$  

$$\begin{equation} s_{x}^{i+1}=s_{x}^{i}+\Delta s_{x}^{i+1} \tag{12}\end{equation}$$  

$$\begin{equation} s_{y}^{i+1}=s_{y}^{i}+\Delta s_{y}^{i+1} \tag{13}\end{equation}$$  


where $v_{x}^{i+1}$ is the x-component of velocity at the $(i+1)’th$ time step, $(t^{i+1})$, $v_{x}^{i}$ is the x-component of velocity at the 
i’th time, $(t^i)$ ,and the time interval $t^{i+1} − t^i = \Delta t$. Similarly for $v_{y}^{i+1}, s_{x}^{i+1}, s_{y}^{i+1}$ etc. The change in velocity and
position after a time $\Delta t$ are given by:

$$\begin{equation} \Delta v_{x}^{i+1}=a_{x}^{i} \Delta t \tag{14}\end{equation}$$

$$\begin{equation} \Delta v_{y}^{i+1}=a_{y}^{i} \Delta t \tag{15}\end{equation}$$

$$\begin{equation} \Delta s_{x}^{i+1}=v_{x}^{i} \Delta t+\frac{1}{2} a_{x}^{i}(\Delta t)^{2} \tag{16}\end{equation}$$  

$$\begin{equation} \Delta s_{y}^{i+1}=v_{y}^{i} \Delta t+\frac{1}{2} a_{y}^{i}(\Delta t)^{2} \tag{17}\end{equation}$$


Where $a^i_x$ and $a^i_y$ are the $x$ and $y$ components of acceleration at the time-step $t^i$.
 
The initial position vector $(s_{x0}, s_{y0})$ and velocity vector $(v_{x0}, v_{y0})$ are specified for time, $t = 0$, and if we know the mass of the planet $M_p$ we can compute the complete spacecraft trajectory.


 ___ 

## 3.  Project Lab:  What you will do

**You will be asked to write code that implements the equations above** to compute the spacecraft trajectory for a total time duration of $t_f$, i.e. from $t = 0$ to $t = t_f$, at time intervals of $\Delta t$.  

**You will use your working trajectory code to**
- Explore some common but critical issues in numerical integrations
- Explore conditions under which your spacecraft can fly by a planet without crashing
- Design trajectories that crash your spacecraft (on purpose) near specific locations on the planet.

**The details are in the project lab notebook.** We will break this down into several pieces.



___ 

## 4. Learning Goals 


- Develop a computer program that performs a 1D numerical simulation of a spacecraft flyby using numerical integration.
> - Understand why planning is a key part of coding by writing pseudo-code to map out how you will implement the desired calculations.
> - Identify which pieces of the problem can/should be coded as reusable functions. 



- Practice writing functions.
> - Write a main function that will return a spacecraft trajectory given initial conditions, a total time for the trajectory and a time step at which the trajectory will be evaluated.  This function will call additional functions (detailed in the project lab notebook).  One of these will check simulation parameters (i.e. inputs) and terminate the code if needed.

    
- Practice "benchmarking" and assessing your code by comparing your results with known answers
> - Compare the outputs of one function with known numerical answers (Part 1).  
> - Compare the outputs of a your functions for one or two time steps with calculations that you do by hand (Part 2).
> - Compare the output for an initial full flyby simulation with a provided figure and a single provided numerical result. (Part 2).
> - Look at your results to assess whether they seem reasonable (Part 3).


- Perform an initial simulation for a flyby of Mercury.
> - Analyze results to determine if they are logically reasonable by comparing them with a previously-validated solution.


- Save reusable functions as a python module that can be reused.
> - Here you will save the main function and its dependencies to a `flyby_fns` module and re-use them by importing the module to do the simulations needed for the following learning goals.


- Investigate the concept of numerical convergence
> - You will investigate the effects of initial conditions (here the initial position and velocity of the sapcecraft) and model resolution (here the time-stepping).


- Design specific flybys (i.e. identify initial conditions) for the following requirements for a mission to Mercury
> - Identify a suite of initial conditions that result in flybys of Mercury for which the closest approach of the spacecraft is within a desired amount of a target altitude. 
> - Design a spacecraft crash-landing at Mercury's south pole to test for the presence of ice. 

 
 ---------------
